In [2]:
from nezha import hiveDb
import sklearn
import numpy as np
import pandas as pd
from pprint import pprint
import timeit
import random
from pyhive import hive
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import plotly as py
import plotly.graph_objs as go
from matplotlib.pylab import style
style.use('ggplot')    
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False  
warnings.simplefilter('ignore')

/opt/conda/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


- 训练集：train.csv
- 验证集1：before_val.csv
- 验证集2：after_val.csv

In [3]:
# 为做 demo 二次筛选特征

In [22]:
%%time
train_data = pd.read_csv('new_train.csv')

CPU times: user 3min 37s, sys: 22.3 s, total: 3min 59s
Wall time: 3min 59s


In [5]:
%%time
null_list = train_data.isnull().sum() / len(train_data)

CPU times: user 1min 13s, sys: 1.13 s, total: 1min 14s
Wall time: 1min 14s


In [9]:
null_list = null_list[null_list>0.8].index.tolist()
np.save('null_list_fornew.npy',null_list)

In [23]:
train_data = train_data.drop(null_list,axis=1)

In [24]:
train_data.to_csv('train_drop0.8null.csv',index=False)

In [25]:
train_data1 = train_data[train_data['target']==1]
train_data2 = train_data[train_data['target']==0].sample(n=20000,random_state=2020)
del train_data
import gc
gc.collect()
train_data = pd.concat([train_data1,train_data2])
train_data.to_csv('sample_train20000_80.csv',index=False)

In [26]:
del train_data1,train_data2
X = train_data.drop('target',axis=1)
y = train_data['target']
del train_data
gc.collect()

7

In [2]:
#train_data = pd.read_csv('sample_train.csv')

In [27]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=2020)

In [28]:
from scipy.stats import ks_2samp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import time
from pprint import pprint
class XGBPipeline():
    def __init__(self,X_train,X_test,y_train,y_test):
        self.X_train = X_train
        self.y_train = y_train       
        self.X_test = X_test    
        self.y_test = y_test
    
    def get_ks(self,y_true,y_pred):
        return ks_2samp(y_pred[y_true==1], y_pred[y_true!=1]).statistic
    
    def timer (func):
        def wrapper(*args,**kwargs): 
            start = time.time()
            result = func(*args,**kwargs)
            end = time.time()
            print(func.__name__+'运行时间：','{:.2f}'.format(end-start))
            return result
        return wrapper
    
    @timer
    def fit_baseline(self):
        print("==========Model BaseLine Train==========")
        params={'booster':'gbtree',
                'objective': 'binary:logistic',
                'max_depth':5,
                'subsample':0.8,
                'colsample_bytree':0.8,
                'min_child_weight':8,
                'learning_rate ': 0.05,
                'nthread':-1,
                'n_estimators':100}
        model = XGBClassifier(**params,verbosity=2)
        print('start')
        model.fit(self.X_train,self.y_train,verbose=True,eval_set=[(self.X_train,self.y_train)],eval_metric='auc')
        self.model_evl(model)
        
    @timer
    def train_model(self,params):
        model = XGBClassifier(**params,verbosity=2)
        model.fit(self.X_train,self.y_train,verbose=2)
        self.model_evl(model)
        return model
        
    @timer
    def gridsearch_para(self):
        params={'booster':'gbtree',
                'objective': 'binary:logistic',
                'max_depth':5,
                'subsample':0.8,
                'colsample_bytree':0.8,
                'min_child_weight':8,
                'learning_rate ': 0.005,
                'nthread':-1,
                'n_estimators':1000}
        print("==========Optimize Parameters==========")
        print("n_estimators:")
        param_test1 = {'n_estimators':range(40,160,20)}
        xgb = XGBClassifier(
            **params,
            scale_pos_weight=float(len(self.y_train.values)-np.sum(self.y_train.values))/float(np.sum(self.y_train.values)),
            seed=2018,
            silent=False)
        gsearch1 = GridSearchCV(estimator = xgb, param_grid = param_test1, scoring='roc_auc',cv=5,n_jobs=-1)
        gsearch1.fit(self.X_train,self.y_train)
        print(gsearch1.best_params_,gsearch1.best_score_)
        params.update(gsearch1.best_params_)
        print("max_depth:")
        param_test2 = {'max_depth':range(3,7,1)}
        xgb = XGBClassifier(
            **params,
            scale_pos_weight=float(len(self.y_train.values)-np.sum(self.y_train.values))/float(np.sum(self.y_train.values)),
            seed=2018,
            silent=False)
        gsearch2 = GridSearchCV(estimator = xgb, param_grid = param_test2, scoring='roc_auc',cv=5,n_jobs=-1)
        gsearch2.fit(self.X_train,self.y_train)
        print(gsearch2.best_params_,gsearch2.best_score_)
        params.update(gsearch2.best_params_)
        print("min_child_weight:")
        param_test3 = {'min_child_weight':range(3,9,1)}
        xgb = XGBClassifier(
            **params,
            scale_pos_weight=float(len(self.y_train.values)-np.sum(self.y_train.values))/float(np.sum(self.y_train.values)),
            seed=2018,
            silent=False)
        gsearch3 = GridSearchCV(estimator = xgb, param_grid = param_test3, scoring='roc_auc',cv=5,n_jobs=-1)
        gsearch3.fit(self.X_train,self.y_train)
        print(gsearch3.best_params_,gsearch3.best_score_)
        params.update(gsearch3.best_params_)
        print("subsample:")
        param_test4 = {
         'subsample':[i/10.0 for i in range(7,10,1)],
        #  'colsample_bytree':[i/10.0 for i in range(6,10,1)]
        }
        xgb = XGBClassifier(
            **params,
            scale_pos_weight=float(len(self.y_train.values)-np.sum(self.y_train.values))/float(np.sum(self.y_train.values)),
            seed=2018,
            silent=False)
        gsearch4 = GridSearchCV(estimator = xgb, param_grid = param_test4, scoring='roc_auc',cv=5,n_jobs=-1)
        gsearch4.fit(self.X_train,self.y_train)
        print(gsearch4.best_params_,gsearch4.best_score_)
        params.update(gsearch4.best_params_)
        print("colsample_bytree:")
        param_test5 = {
          'colsample_bytree':[i/10.0 for i in range(7,10,1)]
        }
        xgb = XGBClassifier(
            **params,
            scale_pos_weight=float(len(self.y_train.values)-np.sum(self.y_train.values))/float(np.sum(self.y_train.values)),
            seed=2018,
            silent=False)
        gsearch5 = GridSearchCV(estimator = xgb, param_grid = param_test5, scoring='roc_auc',cv=5,n_jobs=-1)
        gsearch5.fit(self.X_train,self.y_train)
        print(gsearch5.best_params_,gsearch5.best_score_)
        params.update(gsearch5.best_params_)
        print("调参结束后模型效果:")
        params.update({'scale_pos_weight':float(len(self.y_train.values)-np.sum(self.y_train.values))/float(np.sum(self.y_train.values)),
            'seed':2018,
            'silent':False})
        return self.train_model(params)
        
    def model_evl(self,model):
        print('模型评价：')
        pred = model.predict_proba(self.X_test)
        pred_y = pred[:,1]
        print ('AUC: %.4f' % metrics.roc_auc_score(self.y_test,pred_y))
        ypred = (pred_y>=0.5)*1 
        print ('ACC: %.4f' % metrics.accuracy_score(self.y_test,ypred))
        print ('Recall: %.4f' % metrics.recall_score(self.y_test,ypred))
        print ('Precesion: %.4f' %metrics.precision_score(self.y_test,ypred))
        print ('F1-score: %.4f' %metrics.f1_score(self.y_test,ypred))
        print ('KS: %.4f' %self.get_ks(self.y_test,ypred))
        print('\n')
        print(metrics.confusion_matrix(self.y_test,ypred)) 

In [29]:
xgbp = XGBPipeline(X_train,X_test,y_train,y_test)

In [17]:
xgbp.fit_baseline()

==========Model BaseLine Train==========
start
[0]	validation_0-auc:0.591305
[1]	validation_0-auc:0.606918
[2]	validation_0-auc:0.718001
[3]	validation_0-auc:0.723496
[4]	validation_0-auc:0.737984
[5]	validation_0-auc:0.73947
[6]	validation_0-auc:0.738943
[7]	validation_0-auc:0.750577
[8]	validation_0-auc:0.763221
[9]	validation_0-auc:0.762902
[10]	validation_0-auc:0.766776
[11]	validation_0-auc:0.769425
[12]	validation_0-auc:0.768791
[13]	validation_0-auc:0.772403
[14]	validation_0-auc:0.773784
[15]	validation_0-auc:0.771102
[16]	validation_0-auc:0.774837
[17]	validation_0-auc:0.78159
[18]	validation_0-auc:0.791779
[19]	validation_0-auc:0.810752
[20]	validation_0-auc:0.815685
[21]	validation_0-auc:0.814648
[22]	validation_0-auc:0.824995
[23]	validation_0-auc:0.828255
[24]	validation_0-auc:0.829746
[25]	validation_0-auc:0.844122
[26]	validation_0-auc:0.845126
[27]	validation_0-auc:0.852386
[28]	validation_0-auc:0.858539
[29]	validation_0-auc:0.864655
[30]	validation_0-auc:0.867362
[31]

In [30]:
xgbp.gridsearch_para()

==========Optimize Parameters==========
n_estimators:
{'n_estimators': 140} 0.7794763248524861
max_depth:
{'max_depth': 5} 0.7794763248524861
min_child_weight:
{'min_child_weight': 3} 0.7706818554555852
subsample:
{'subsample': 0.9} 0.7759738434051251
colsample_bytree:
{'colsample_bytree': 0.7} 0.7794647841963543
调参结束后模型效果:
模型评价：
AUC: 0.7878
ACC: 0.9607
Recall: 0.2986
Precesion: 0.3879
F1-score: 0.3374
KS: 0.2822


[[7887  131]
 [ 195   83]]
train_model运行时间： 101.95
gridsearch_para运行时间： 1330.88
